In [2]:
import re

import pandas as pd

In [5]:
df = pd.read_csv("../data/input/dados_ficha_a_desafio.csv")
print(f"Visualização inicial do dataframe:")
df.head(3)

Visualização inicial do dataframe:


,id_paciente,sexo,obito,bairro,raca_cor,ocupacao,religiao,luz_eletrica,data_cadastro,escolaridade,...,familia_beneficiaria_auxilio_brasil,crianca_matriculada_creche_pre_escola,altura,peso,pressao_sistolica,pressao_diastolica,n_atendimentos_atencao_primaria,n_atendimentos_hospital,updated_at,tipo
0,cd6daa6e-038d-4952-af29-579e62e07f97,male,0,Parada de Lucas,Branca,Não se aplica,Sem religião,True,2021-06-24 00:00:00.000,Fundamental Incompleto,...,0,0,172.0,52.5,110.0,70.0,8,9,2021-06-01 00:00:00.000,historico
1,ad6cecb2-3a44-49ab-b2f3-6f9ffc2e2ec7,male,0,Cidade Nova,Parda,Assistente Administrativo,Evangélica,1,2021-01-07 00:00:00.000,Médio Incompleto,...,1,0,158.0,76.2,140.0,80.0,0,6,2020-06-25 00:00:00.000,historico
2,54e834e7-e722-4daa-8909-cf917a1247e6,male,0,Santa cruz,Parda,Não se aplica,Católica,1,2021-02-18 00:00:00.000,Fundamental Completo,...,0,0,53.0,82.5,180.0,60.0,5,2,2020-03-02 00:00:00.000,historico


In [6]:
print(f"Colunas do dataframe:")
print(df.columns)

Colunas do dataframe:
Index(['id_paciente', 'sexo', 'obito', 'bairro', 'raca_cor', 'ocupacao',
       'religiao', 'luz_eletrica', 'data_cadastro', 'escolaridade',
       'nacionalidade', 'renda_familiar', 'data_nascimento',
       'em_situacao_de_rua', 'frequenta_escola', 'meios_transporte',
       'doencas_condicoes', 'identidade_genero', 'meios_comunicacao',
       'orientacao_sexual', 'possui_plano_saude', 'em_caso_doenca_procura',
       'situacao_profissional', 'vulnerabilidade_social',
       'data_atualizacao_cadastro', 'familia_beneficiaria_auxilio_brasil',
       'crianca_matriculada_creche_pre_escola', 'altura', 'peso',
       'pressao_sistolica', 'pressao_diastolica',
       'n_atendimentos_atencao_primaria', 'n_atendimentos_hospital',
       'updated_at', 'tipo'],
      dtype='object')


# Coluna: id_paciente

## Requisito: Unicidade

In [7]:
print("Avaliando se todas as entradas de 'id_paciente' são únicas:")
id_counts = df["id_paciente"].value_counts()
id_counts_counts = id_counts.value_counts()
print(id_counts_counts)

Avaliando se todas as entradas de 'id_paciente' são únicas:
count
1    98007
2      986
3        7
Name: count, dtype: int64


In [8]:
print(f"Há {id_counts_counts[2]} id's duplicados e {id_counts_counts[3]} id's triplicados")
id_triplicado_exemplo = id_counts[id_counts == 3].index[0]
print(f"Mostrando todas as ocorrências para o id triplicado '{id_triplicado_exemplo}'")
df[df["id_paciente"] == id_triplicado_exemplo]

Há 986 id's duplicados e 7 id's triplicados
Mostrando todas as ocorrências para o id triplicado '49cf308b-411f-4fd5-a33d-0446d48de446'


,id_paciente,sexo,obito,bairro,raca_cor,ocupacao,religiao,luz_eletrica,data_cadastro,escolaridade,...,familia_beneficiaria_auxilio_brasil,crianca_matriculada_creche_pre_escola,altura,peso,pressao_sistolica,pressao_diastolica,n_atendimentos_atencao_primaria,n_atendimentos_hospital,updated_at,tipo
42083,49cf308b-411f-4fd5-a33d-0446d48de446,female,0,Catumbi,Branca,Técnico Mecânico,Religião de matriz africana,1,2023-08-29 10:59:09.410,Fundamental Incompleto,...,0,0,117.0,80.20,120.0,70.0,0,12,2020-03-02 00:00:00.000,historico
48094,49cf308b-411f-4fd5-a33d-0446d48de446,female,0,Irajá,Branca,Não se aplica,Outra,1,2021-03-24 19:35:47,Superior completo,...,0,1,158.0,70.85,170.0,100.0,0,7,2020-09-02 00:00:00.000,historico
87084,49cf308b-411f-4fd5-a33d-0446d48de446,female,False,Alto da Boa Vista,Parda,Empregado doméstico nos serviços gerais,Sem religião,1,2022-05-11 08:45:32.933,Fundamental Incompleto,...,0,1,155.0,18.60,143.0,70.0,0,3,2022-05-13 00:00:00.000,historico


In [9]:
print("Avaliando se há algum id duplicado/triplicado com repetição dos dados")
duplicated_ids = id_counts[id_counts > 1].index
duplicated_entries = df[df["id_paciente"].isin(duplicated_ids)]
grouped = duplicated_entries.groupby("id_paciente").nunique()
grouped

# Checando se há entradas duplicadas/triplicadas com todos os dados iguais
equal_entries = grouped[grouped.eq(1).all(axis=1)]
print(f"Total de entradas duplicadas/triplicadas com todos os dados iguais: {equal_entries.shape[0]}")
print(f"Avaliando quão semelhante os id's repetidos são (caso sejam leves modificações dos mesmos pacientes):")
entries_similarity = grouped.eq(1).mean(axis=1).sort_values(ascending=False).head(10)
print(f"Valor máximo da média de valores iguais entre os id's duplicados/triplicados: {entries_similarity.max():.2%}")
print(f"Avaliação final: os id's duplicados/triplicados são de pacientes diferentes")

Avaliando se há algum id duplicado/triplicado com repetição dos dados
Total de entradas duplicadas/triplicadas com todos os dados iguais: 0
Avaliando quão semelhante os id's repetidos são (caso sejam leves modificações dos mesmos pacientes):
Valor máximo da média de valores iguais entre os id's duplicados/triplicados: 55.88%
Avaliação final: os id's duplicados/triplicados são de pacientes diferentes


## Requisito: Máscara

In [10]:
def verificar_mascara_id_paciente(id_paciente: str) -> bool:
    """
    Verifica se a máscara do id_paciente está correta.
    """
    return bool(re.match(r"^[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{12}$", id_paciente))

In [11]:
print(f"Todos os id's respeitam a máscara? {str(df["id_paciente"].apply(verificar_mascara_id_paciente).all())}")

Todos os id's respeitam a máscara? True


# Coluna: sexo

In [12]:
print(f"Valores únicos da coluna 'sexo': {df["sexo"].unique()}")
print(f"Está ok, porém o dado está em inglês, enquanto o resto da base está em português.")

Valores únicos da coluna 'sexo': ['male' 'female']
Está ok, porém o dado está em inglês, enquanto o resto da base está em português.


# Coluna: obito

In [13]:
print(f"Valores únicos da coluna 'obito': {df["obito"].unique()}")
print("O dado está inconsistente, variando entre (0, 1) e (False, True)")
print("Isto deve ser corrigido para ficar consistente.")

Valores únicos da coluna 'obito': ['0' 'False' '1' 'True']
O dado está inconsistente, variando entre (0, 1) e (False, True)
Isto deve ser corrigido para ficar consistente.


# Coluna: bairro

In [14]:
print(f"Valores únicos da coluna 'bairro': {df["bairro"].unique()[:10]}")

Valores únicos da coluna 'bairro': ['Parada de Lucas' 'Cidade Nova' 'Santa cruz' 'Bangu' 'Santíssimo'
 'Jardim Vinte e Cinco de Agosto' 'Penha Circular' 'Guadalupe'
 'Manguinhos' 'Irajá']


In [15]:
def salvar_count_unique(
    series: pd.Series,
    path: str,
    ascending: bool = True,
    verbose: bool = False
    ) -> None:
    """
    Salva a contagem de valores únicos de uma série em um arquivo CSV. O foco 
    desta função é auxiliar a encontrar valores errados/problemáticos em uma
    série cheia de valores únicos.
    
    Os valores únicos são ordenados de acordo com a contagem de ocorrências, e
    então salvos em um arquivo CSV para serem avaliados, dando prioridade aos
    valores com menor ocorrência. Isto é feito porque se assume que os valores
    com uma ocorrência média/alta são menos sucetíveis a serem erros.
    
    Isto deve ser feito para economizar esforço. Avaliar todos os valores únicos
    da série é uma abordagem mais eficaz (porém menos eficiente).

    Parameters
    ----------
    series : pd.Series
        A série com os valores (não únicos) a serem avaliados.
    path : str
        O caminho para salvar o arquivo CSV.
    ascending : bool, optional
        Se os valores devem ser ordenados em ordem crescente. O padrão é True.
    verbose : bool, optional
        Se deve imprimir mensagens de log. O padrão é False.
    """
    if verbose:
        print(f"Arquivo com contagem salvo em: {path}")
    series.value_counts().sort_values(ascending=ascending).to_csv(path)

In [16]:
print(f"Avaliando os bairros com menor quantidade de ocorrências")
local_bairro_counts = "../data/output/bairro_counts.csv"
salvar_count_unique(df["bairro"], local_bairro_counts, verbose=True)
print("Foram avaliados os nomes dos bairros, focando nos bairros com menor ocorrência")
print("Isso porque se assume que caso um bairro tenha alta quantidade de ocorrências, dificilmente será um dado errado")
print("Não foi identificado nenhum bairro com nome problemático")

Avaliando os bairros com menor quantidade de ocorrências
Arquivo com contagem salvo em: ../data/output/bairro_counts.csv
Foram avaliados os nomes dos bairros, focando nos bairros com menor ocorrência
Isso porque se assume que caso um bairro tenha alta quantidade de ocorrências, dificilmente será um dado errado
Não foi identificado nenhum bairro com nome problemático


# Coluna: raca_cor

In [17]:
print(f"Valores únicos da coluna 'raca_cor': {df["raca_cor"].unique()}")
print(f"Uma das ocorrências é 'Não' ({df['raca_cor'].value_counts()['Não']} ocorrências)")
print("Isto parece errado, com o dado equivalente esperado sendo 'Não Informado'")

Valores únicos da coluna 'raca_cor': ['Branca' 'Parda' 'Preta' 'Amarela' 'Não' 'Indígena']
Uma das ocorrências é 'Não' (3 ocorrências)
Isto parece errado, com o dado equivalente esperado sendo 'Não Informado'


# Coluna: ocupacao

In [23]:
print(f"Valores únicos da coluna 'ocupacao': {df["ocupacao"].unique()}")
salvar_count_unique(df["ocupacao"], "../data/output/ocupacao_counts.csv", verbose=True)
print("Diversas entradas possuem o token \". Isto não fica aparente no python, mas deve ser corrigido")

Valores únicos da coluna 'ocupacao': ['Não se aplica' 'Assistente Administrativo'
 'Representante Comercial Autônomo' ... 'Assistente de Coreografia'
 'Produtor de Espécies Frutíferas Rasteiras'
 'Supervisor da confecção de artefatos de tecidos, couros e afins']
Arquivo com contagem salvo em: ../data/output/ocupacao_counts.csv
Diversas entradas possuem o token ". Isto não fica aparente no python, mas deve ser corrigido


# Coluna: religiao

In [31]:
print(f"Valores únicos da coluna 'religiao': {df["religiao"].unique()}")
print("\nContagem das ocorrências de cada valor:\n")
print(df["religiao"].value_counts())

Valores únicos da coluna 'religiao': ['Sem religião' 'Evangélica' 'Católica' 'Outra' 'Espírita'
 'Religião de matriz africana' 'Não' 'Budismo' 'Judaísmo' 'ESB ALMIRANTE'
 'Candomblé' 'Islamismo' '10 EAP 01' 'Acomp. Cresc. e Desenv. da Criança'
 'ORQUIDEA' 'Sim']

Contagem das ocorrências de cada valor:

religiao
Sem religião                          35119
Católica                              28348
Outra                                 18714
Evangélica                            14798
Espírita                               1974
Religião de matriz africana             731
Não                                     174
Budismo                                  81
Judaísmo                                 32
Islamismo                                 8
Acomp. Cresc. e Desenv. da Criança        7
Candomblé                                 6
ESB ALMIRANTE                             5
10 EAP 01                                 1
ORQUIDEA                                  1
Sim                       

In [41]:
print("Aqui ocorre novamente o caso de haver o valor 'Não' ao invés de 'Não Informado', ou neste caso, 'Sem religião'.")
print("Dificilmente será possível recuperar este dado")
print("Deve ser decidido por um especilista de negócio qual o valor padrão")
print("Valores padrão sugeridos: 'Não Informado'")
print("\nHá diversas ocorrências que não aparentam ser religiões conhecidas:")
print(f"{[
    'ESB ALMIRANTE', '10 EAP 01', 'Acomp. Cresc. e Desenv. da Criança',
    'ORQUIDEA', 'Sim'
]}")
count_religioes_nao_reconhecidas = df["religiao"].isin([
    'ESB ALMIRANTE', '10 EAP 01', 'Acomp. Cresc. e Desenv. da Criança',
    'ORQUIDEA', 'Sim'
]).sum()
print(f"Como estes valores são poucos ({count_religioes_nao_reconhecidas}), é razoável mudál-los para 'Outra'")

Aqui ocorre novamente o caso de haver o valor 'Não' ao invés de 'Não Informado', ou neste caso, 'Sem religião'.
Dificilmente será possível recuperar este dado
Deve ser decidido por um especilista de negócio qual o valor padrão
Valores padrão sugeridos: 'Não Informado'

Há diversas ocorrências que não aparentam ser religiões conhecidas:
['ESB ALMIRANTE', '10 EAP 01', 'Acomp. Cresc. e Desenv. da Criança', 'ORQUIDEA', 'Sim']
Como estes valores são poucos (15), é razoável mudál-los para 'Outra'


In [40]:
df["religiao"].isin([
    'ESB ALMIRANTE', '10 EAP 01', 'Acomp. Cresc. e Desenv. da Criança',
    'ORQUIDEA', 'Sim'
]).sum()

np.int64(15)

# Coluna: luz_eletrica

In [43]:
print(f"Valores únicos da coluna 'luz_eletrica': {df["luz_eletrica"].unique()}")
print("Novamente, há dois padrões diferentes para representar a mesma informação")
print("As ocorrências no padrão (0, 1) devem ser convertidas para (False, True)")

Valores únicos da coluna 'luz_eletrica': ['True' '1' '0' 'False']
Novamente, há dois padrões diferentes para representar a mesma informação
As ocorrências no padrão (0, 1) devem ser convertidas para (False, True)
